# PA005: High Value Customer Identification

# 0.0. Planejamento da Solução ( IOT )

## Input - Entrada

### O problema de negócio

1. Selecionar os mais valiosos clientes para formar o programa de fidelidade **"INSIDERS"**
2. Conjunto de dados com as vendas de produtos durante 1 ano ( Nov 2015 to Dec 2017 )

## Output - Saída

1. A indicação de pessoas para fazer parte do programa de fidelidade **"INSIDERS"**.
2. Relatório com as respostas para as seguintes perguntas:
    - Quem são as pessoas elegíveis para participar do programa de Insiders ?
    - Quantos clientes farão parte do grupo?
    - Quais as principais características desses clientes ?
    - Qual a porcentagem de contribuição do faturamento, vinda do Insiders ?
    - Qual a expectativa de faturamento desse grupo para os próximos meses ?
    - Quais as condições para uma pessoa ser elegível ao Insiders ?
    - Quais as condições para uma pessoa ser removida do Insiders ?
    - Qual a garantia que o programa Insiders é melhor que o restante da base ?
    - Quais ações o time de marketing pode realizar para aumentar o faturamento?

## Tasks - Processo

1. **Quem são as pessoas elegíveis para participar do programa de Insiders ?**
    - O que é ser elegível ? O que é um cliente "valioso" para a empresa ?
        - Faturamento:
            - Alto Ticket Médio
            - Alto LTV
            - Baixa Recência ou Alta Frequência ( tempo entre as compras )
            - Alto Basket Size ( quantidade média de produtos comprados )
            - Baixa probabilidade de Churn
            - Previsão alta de LTV
            - Alta propensão de compra

        - Custo:
            - Baixo número de devoluções

        - Experiência:  
            - Média alta de avaliações
            
            
2. **Quantos clientes farão parte do grupo?**
    - Número de clientes
    - % em relação ao total de clients
    
    
3. **Quais as principais características desses clientes ?**
    - Escrever os principais atributos dos clientes
        - Idade
        - País
        - Salário
        
    - Escrever os principais comportamentos de compra dos clients ( métricas de negócio )
        - Vide acima
    
    
4. **Qual a porcentagem de contribuição do faturamento, vinda do Insiders ?**
    - Calcular o faturamento total da empresa durante o ano.
    - Calcular o faturamento (%) apenas do cluster Insiders.
    
    
5. **Qual a expectativa de faturamento desse grupo para os próximos meses ?**
    - Cálculo do LTV do grupo Insiders
    - Séries Temporais ( ARMA, ARIMA, HoltWinter, etc )
    

6. **Quais as condições para uma pessoa ser elegível ao Insiders ?**
    - Qual o período de avaliação ?
    - O "desempenho" do cliente está próximo da média do cluster Insiders. 
    
    
7. **Quais as condições para uma pessoa ser removida do Insiders ?**
    - O "desempenho" do cliente não está mais próximo da média do cluster Insiders. 
    
    
8. **Qual a garantia que o programa Insiders é melhor que o restante da base ?**
    - Teste de Hipóteses
    - Teste A/B
    
    
9. **Quais ações o time de marketing pode realizar para aumentar o faturamento?**
    - Descontos
    - Preferências de escolha
    - Produtos exclusivos

# 0.0. Imports

In [4]:
import numpy   as np
import pandas  as pd
import seaborn as sns

from matplotlib import pyplot as plt
from IPython.display import HTML

from sklearn import cluster as c
from sklearn import metrics as m

from plotly import express as px

## 0.1. Helper Functions

In [5]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'ggplot')
    plt.rcParams['figure.figsize'] = [24, 9]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()
    
jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


## 0.2. Load dataset

In [9]:
!cd

C:\Users\Arbo\repos\my_git\insiders-clustering\notebooks


In [18]:
# load data
df_raw = pd.read_csv( r'C:\Users\Arbo\repos\my_git\insiders-clustering\data\raw\Ecommerce.csv', encoding= 'unicode_escape')

# drop extra column
df_raw = df_raw.drop( columns=['Unnamed: 8'], axis=1 )

# <font color='black'> 1.0. Descrição dos dados </font>

In [25]:
df1 = df_raw.copy()

## 1.1. Rename columuns

In [26]:
cols_new = ['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date',
             'unit_price', 'customer_id', 'country']

In [27]:
df1.columns = cols_new

## 1.2. Data dimensions

In [28]:
print('Number of rows: {}'.format(df1.shape[0]))
print('Number of cols: {}'.format(df1.shape[1]))

Number of rows: 541909
Number of cols: 8


## 1.3. Data types

Attributes Desciption

* invoice_no   -> número referente a transação do cliente, valor único para cada transação

* stock_code   -> código do item

* description  -> descrição do item

* quatity      -> quantidade de cada item por transação

* invoice_date -> data que a transação foi gerada

* unit_price   -> preço unitário do produto

* customer_id  -> ID do ciente

* country      -> localizaão do cliente 

In [33]:
df1.head(3)

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,29-Nov-16,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0,29-Nov-16,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,29-Nov-16,2.75,17850.0,United Kingdom


In [34]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity        float64
invoice_date     object
unit_price      float64
customer_id     float64
country          object
dtype: object

## 1.4. Check NA

In [30]:
df1.isnull().sum()

invoice_no           0
stock_code        6035
description       7489
quantity          6035
invoice_date      6035
unit_price        6035
customer_id     138727
country           6035
dtype: int64

## 1.5. Replace NA

In [37]:
df1 = df1.dropna(subset=['description','customer_id'])

print('Removed data: {:.2f}'.format(1-(df1.shape[0]/df_raw.shape[0])))

Removed data: 0.26


In [38]:
df1.shape

(403182, 8)

In [39]:
df1.isnull().sum()

invoice_no      0
stock_code      0
description     0
quantity        0
invoice_date    0
unit_price      0
customer_id     0
country         0
dtype: int64

## 1.6. Change dtypes

In [40]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity        float64
invoice_date     object
unit_price      float64
customer_id     float64
country          object
dtype: object

In [42]:
# invoice_date
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'], format='%d-%b-%y') #format é o formato origiral que está no dataset, para sim a função realizar a formatação

# customer_id
df1['customer_id'] = df1['customer_id'].astype('int64')

In [43]:
df1.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2016-11-29,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0,2016-11-29,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,2016-11-29,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2016-11-29,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,2016-11-29,3.39,17850.0,United Kingdom


## 1.7. Descriptive Statistics

Realização desta etapa em outro momento do ciclo. Pois o objetivo é rodar o modelo, antes de análisar os dados.

# <font color='black'> 2.0. Feature Engineering </font>

In [45]:
df2 = df1.copy()

## 2.1. Feature Creation

Criação das variáveis base, em relação ao modelo RFM utilizado com Benchmark para o projeto.

In [48]:
# data reference - tabela para chaves primarias

df_ref = df2.drop(['invoice_no','stock_code','description','quantity',
                   'invoice_date','unit_price','country'], axis=1).drop_duplicates(ignore_index = True)

df_ref.head()

,customer_id
0,17850
1,13047
2,12583
3,13748
4,15100


# <font color='red'> 3.0. Filtragem de Varivéis </font>

# <font color='red'> 4.0. EDA ( Exploratory Data Analysis ) </font>

# <font color='red'> 5.0. Data Preparation </font>

# <font color='red'> 6.0. Feature Selection </font>

# <font color='red'> 7.0. Hyperparameter Fine-Tunning </font>

# <font color='red'> 8.0. Model Training </font>

## 8.1. K-Means

### 8.2. Cluster Validation

# <font color='red'> 9.0. Cluster Analysis </font>

## 9.1. Visualization Inspection

## 9.2. Cluster Profile

### Cluster 01:  ( Candidato à Insider )
    - Número de customers: 6 (0.14% do customers )
    - Recência em média: 7 dias
    - Compras em média: 89 compras
    - Receita em média: $182.182,00 dólares
        
### Cluster 02: 
    - Número de customers: 31 (0.71 do customers )
    - Recência em média: 14 dias
    - Compras em média: 53 compras
    - Receita em média: $40.543,52 dólares
        
### Cluster 03: 
    - Número de customers: 4.335 (99% do customers )
    - Recência em média: 92 dias
    - Compras em média: 5 compras
    - Receita em média: $1.372,57 dólares

# <font color='red'> 10.0. Deploy To Production </font>